In [88]:
import parflow as pf
import numpy as np
import pandas as pd
import xarray as xr
import plotly.express as px

In [89]:
num_timesteps = 10
dump_interval = 1

runs_folder = "./domains/outputs"
run_dir = runs_folder + "/conus2_mjb_conus2_2005WY"
run_name = "conus2_mjb"
data_accessor = pf.Run.from_definition(f'{run_dir}/{run_name}.pfidb').data_accessor
run_def = {}
run_def["mask"] = np.where(data_accessor.mask > 0, 1, np.nan)
run_def["porosity"] = data_accessor.computed_porosity
run_def["dx"] = data_accessor.dx
run_def["dy"] = data_accessor.dy
run_def["dz"] = data_accessor.dz
run_def["specific_storage"] = data_accessor.specific_storage
mask = pf.read_pfb(run_dir + "/mask.pfb")


Solver: Field BinaryOutDir is not part of the expected schema <class 'parflow.tools.database.generated.Solver'>
  - nt
  - sw_ini
  - hkdepth
  - wtfact
  - trsmx0
  - smpmax
  - pondmx


In [90]:
def load_pfb(variable, time, dump_interval=1):
    timestamp = f"{int(time/dump_interval)}".rjust(5, "0")
    return pf.read_pfb(run_dir + f"/{run_name}.out.{variable}.{timestamp}.pfb")

In [91]:
def load_variable(loading_function, time, data_accessor):
    data_accessor.time = time
    return loading_function(data_accessor)

In [92]:

data = pd.DataFrame()
data["time"] = np.arange(0, num_timesteps*dump_interval, dump_interval)
data["pressure"] = np.array(load_variable(lambda x: data_accessor.pressure, time, data_accessor)*run_def["mask"] for time in data.time)

data["pressure2"] = np.array(load_pfb("press", time, dump_interval)*run_def["mask"] for time in data.time)
data["saturation"] = np.array(load_variable(lambda x: data_accessor.pressure, time, data_accessor)*run_def["mask"] for time in data.time)
data["subsurface_storage"] = np.array(load_variable(lambda x: data_accessor.subsurface_storage, time, data_accessor)*run_def["mask"] for time in data.time)
data["surface_storage"] = np.array(load_variable(lambda x: data_accessor.surface_storage, time, data_accessor)*run_def["mask"][0] for time in data.time)

In [93]:
data.surface_storage[0].sum()

nan

In [97]:
px.imshow(data["surface_storage"][0]-data["surface_storage"][9])

In [ ]:
px.imshow(run_def["mask"][0])